In [1]:
import numpy as np
import os, sys, glob, abc
%matplotlib inline
from matplotlib import pyplot as plt, colors
def read(filelist):
    data = []
    for f in sorted(filelist):
        x = np.load(f)
        if len(data) == 0: data = x.copy()
        else: data = np.concatenate([data, x])
    return data

# Where is the dataset stored?
dataset = "/data/i3store/users/mjlarson/student_data"

# Read in all of the data files
data_files = dataset + "/IC86_*exp.npy"
data = read(glob.glob(data_files))

# Read in all of the MC files 
sim_files = dataset + "/IC86*MC*npy"
sim = read(glob.glob(sim_files))

# Set the angular error floor to 0.2 degrees
data['angErr'][data['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
sim['angErr'][sim['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)

data['ra']=np.random.uniform(0,2*np.pi,size=len(data))


Bad key "text.kerning_factor" on line 4 in
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
import mla.core
from mla.spectral import *
import threeML
#sp= PowerLaw(1,1e-15,2)
sp=threeML.Powerlaw()


/data/disk01/home/jasonfan/threeML/threeML/__init__.py:12: UserWarning: No DISPLAY variable set. Using backend for graphics without display (Agg)
  warnings.warn("No DISPLAY variable set. Using backend for graphics without display (Agg)")
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/astromodels/core/parameter.py:555: UserWarning: We have set the min_value of K to 1e-99 because there was a postive transform
  warnings.warn('We have set the min_value of %s to 1e-99 because there was a postive transform' % self.path)
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/astromodels/core/parameter.py:555: UserWarning: We have set the min_value of xc to 1e-99 because there was a postive transform
  warnings.warn('We have set the min_value of %s to 1e-99 because there was a postive transform' % self.path)

WARNING UserWarning: Using default configuration from /data/disk01/home/jasonfan/threeML/threeML/data/threeML_config.yml. You might want to

In [4]:
from mla.spectral import *
from mla import injection
sp= PowerLaw(1,1e-15,2)
dec = 22.0145*np.pi/180
ra = 83.63*np.pi/180
inj = injection.PSinjector(sp,sim)
inj.set_source_location(ra,dec)
sample=inj.sample_nevents(15)
background = np.random.choice(data, 30).copy()
background['time']=0.5

In [5]:
sample['dec']*180/np.pi

array([21.79288505, 22.04000879, 21.55584384, 21.57397131, 21.09150297,
       22.08663683, 21.9866074 , 22.82114075, 22.10406364, 22.01687756,
       22.52742871, 21.87872226, 22.72580831, 23.62963202, 22.68719934])

In [6]:
import numpy.lib.recfunctions as rf
sample = rf.drop_fields(sample, [n for n in sample.dtype.names \
     if not n in background.dtype.names])
injected = np.concatenate([background,sample])
np.save("testdata.npy",injected)

In [ ]:
from imp import reload
import scipy.optimize
reload(mla.core)
#sp.K=1e-8
#sp.index=-2
sp= PowerLaw(1,1e-15,2)
testdata=np.load("testdata.npy")
import warnings
warnings.filterwarnings("ignore")
test=mla.core.LLH_point_source(np.radians(83.63) ,np.radians(22.0145) , data , sim , 300 , sp)
test.update_data(testdata,1000/3600/24)
#test.add_injection(sample)


In [ ]:
def fit(ns):
    return -test.eval_llh_ns(ns)[1]
result=scipy.optimize.minimize(fit,5,bounds=[(0,test.N)])

In [ ]:
result

In [ ]:
x=np.linspace(0,test.N,100)
temp=[]
ns_true = (testdata['run']>200000).sum()
for i in x:
    temp.append(fit(i))

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt, colors
fig, ax = plt.subplots(figsize=(10,6))
ax.axvline(result.x,
            color='b',
           linewidth=3,
            label = 'Best fit from scipy')
ax.axvline(ns_true,
            color='k',
           linewidth=3,
            label = 'True number of signal events')
ax.plot(x,temp,label = 'gamma '+str(-2))
ax.grid(alpha=0.3)
ax.set_xlim(0, len(testdata))
ax.legend(fontsize=16,)
ax.set_xlabel("ns", fontsize=16)
ax.set_ylabel("TS", fontsize=16)
ax.tick_params(which='both', labelsize=16)
ax.set_title("Result from mla", fontsize=24)

In [ ]:
bins = np.array([np.linspace(-1, 1, 300), # bins for sin(dec)
                 np.linspace(1, 8, 100)]) # bins for logE


# And draw it.
fig, ax = plt.subplots(figsize=(11,6))
cmesh = ax.pcolormesh(bins[0], bins[1],
                      test.ratio.T,
                      cmap = 'RdBu',
                      norm=colors.LogNorm(),
                      vmin=1e-2,
                      vmax=1e2)
cbar = plt.colorbar(cmesh, ax=ax)

ax.grid(alpha=0.2)
ax.set_xlabel("Log(E)", fontsize=16)
ax.set_ylabel(r"sin($\delta$)", fontsize=16)
ax.tick_params(which='both', labelsize=14)
cbar.ax.set_ylabel("Signal/Background", fontsize=16)
cbar.ax.tick_params(labelsize=14)

In [ ]:
test.spatial

In [ ]:
testdata['dec']*180/np.pi